In [1]:
# split graph into specific nodes per subject.

import ansible_vault
import json
import pathlib
import pydash
import rdflib

graph_in = rdflib.Graph()

for x in [x for x in (pathlib.Path.cwd().parents[0] / 'rdf').iterdir() if x.suffix == '.json']:
    graph_in.parse(x)

ont = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/ontology/")
res = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/")

graph_in.bind("ont", ont)
graph_in.bind("res", res)

with open(pathlib.Path.cwd().parents[0] / "authentication" / "keys.json") as keys:
    keys = json.load(keys)

subjects = list()

openstate = [
    s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.OpenStatement))
]

for s, p, o in graph_in.triples((None, None, None)):
    if s in openstate and p == ont.hasPayload:
        fragment = rdflib.Graph()
        fragment.parse(data=o)
        for a, b, c in fragment.triples((None, None, None)):
            subjects.append(a)

closedstate = [
    s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.ClosedStatement))
]

for s, p, o in graph_in.triples((None, None, None)):
    if s in closedstate and p == ont.hasPayload:
        fragment = rdflib.Graph()
        unencruptr = ansible_vault.Vault(keys[str(s)]).load_raw(o).decode()
        fragment.parse(data=unencruptr)
        for a, b, c in fragment.triples((None, None, None)):
            subjects.append(a)

subjects = pydash.uniq(subjects)
subjects = [x for x in subjects if 'well-known' not in str(x)]

print(len(subjects)) # okay we have five files to write

# next phase is that we decrypt the full graph and determine which statements belong to the subject, or are linked by blank nodes to the subject.

for x in subjects:
    print(x)


5
https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/713527d4-9feb-4015-be1d-6d3dd68b0e89
https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/d4b77081-311b-4a44-b388-6dfb826faa0d
https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/1f220116-86a8-4818-ae50-c1c39bbb06e9
https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/94efddce-3f2a-4838-a5fa-fcf4180e5584
https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/8605ab63-d1d0-4324-8997-8c33de2c3673
